In [1]:
import os
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [2]:

# RL starts here

import gymnasium
from gymnasium.wrappers import RecordVideo

from stable_baselines3 import PPO # Proximal Policy Optimization
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

from pathlib import Path

In [3]:
# Create the LunarLander environment for training
env = gymnasium.make("LunarLander-v3")

/home/ubuntu/repos/hf-rl/.venv/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [ ]:
# Create a fresh environment for training (must be new each time you run this cell)
env = gymnasium.make("LunarLander-v3")

# Create the policy model.
model = PPO(
    policy="MlpPolicy",
    env=env,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1,
    device='cpu'
)

# Train the PPO model.
model.learn(total_timesteps=1000000)

# Eval (higher reward is better)
eval_env = Monitor(gymnasium.make("LunarLander-v3"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 89.7     |
|    ep_rew_mean     | -186     |
| time/              |          |
|    fps             | 1346     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 1024     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 88.4          |
|    ep_rew_mean          | -172          |
| time/                   |               |
|    fps                  | 562           |
|    iterations           | 2             |
|    time_elapsed         | 3             |
|    total_timesteps      | 2048          |
| train/                  |               |
|    approx_kl            | 0.00023861969 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2       

In [ ]:
model_name = "ppo-LunarLander-v3"
model.save(model_name)

In [9]:
# Create a fresh environment for video recording
video_folder = Path("./videos")
video_folder.mkdir(exist_ok=True)

record_env = gymnasium.make("LunarLander-v3", render_mode="rgb_array")
video_env = RecordVideo(record_env, video_folder=str(video_folder), episode_trigger=lambda x: True)

obs, info = video_env.reset()

for _ in range(3):
    done = False
    while not done:
        # Get observation of environment, tell model to take an action.
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = video_env.step(action)
        done = terminated or truncated  # Either out of bounds or we crashed.
    obs, info = video_env.reset()

video_env.close()
print("Videos saved to ./videos/")

NameError: name 'model' is not defined